Chunks generated by [MinerU](https://github.com/opendatalab/MinerU) are stored inside `output/{file_name}` . 
- This script will do some preprocessing and save the chunks to `output/{file_name}/data.csv` file

In [16]:
import json,csv

In [18]:
FOLDER_NAME = "Raspberry_short_vers"

folder_path = f"output/{FOLDER_NAME}"
image_path = f"{folder_path}/images"
content_lst = f"{folder_path}/{FOLDER_NAME}_content_list.json"
csv_file = f"{folder_path}/{FOLDER_NAME}_csv.csv"


In [19]:
with open(content_lst,"r") as f:
    content_lst_obj = json.loads(f.read())

fin_cont = []
for content_lst in content_lst_obj:
    page_num = content_lst["page_idx"]+1
    if content_lst["type"] == "image":
        fin_cont.append(["",page_num,content_lst["img_path"]])        
    elif content_lst["type"] == "text":
        txt = content_lst["text"]
        if "text_level" in content_lst:
            txt=("#"*content_lst["text_level"])+txt
        fin_cont.append([txt,page_num,""])

with open(csv_file, 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerows(fin_cont)
        